In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm_notebook
import tqdm
# import torchaudio
from sklearn.model_selection import train_test_split
import glob
import os
import sys
import librosa
import IPython.display as ipd
import wave

In [2]:
import torch
print(torch.version.cuda) #: Returns CUDA version of the currently installed packages
print(torch.cuda.is_available()) #: Returns True if CUDA is supported by your system, else False
# current_device = torch.cuda.current_device()#: Returns ID of current device
# print(current_device)
# print(torch.cuda.get_device_name(current_device)) #: Returns name of the CUDA device with ID = ‘device_ID’


c:\Users\Demian\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


None
False


In [4]:
Iemocap_path = 'G:\Arete\projects\SER_project\datasets\IEMOCAP\Iemocap_audio\iemocap_audio\IEMOCAP_wav\*'
df_path = 'G:\Arete\projects\SER_project\datasets\\testfolder\iemocap.csv'
save_data_path = "G:\Arete\projects\SER_project\datasets\\testfolder\zc"


In [8]:
def Iemocap_dataset(path):
  data = []
  all_folders_directory = glob.glob(path)

  for wave_file in all_folders_directory:
    if '_F'  not in wave_file and'_M' not in wave_file:  # to get tha complete conversation records
    # if '_F'  in wave_file or'_M' in wave_file: # to get the subrecords of the conversations records
      data.append({"name":wave_file})

  df = pd.DataFrame(data)
  return df

Iemocap_df = Iemocap_dataset(Iemocap_path)
print(Iemocap_df.shape)
Iemocap_df.head()

(151, 1)


,name
0,G:\Arete\projects\SER_project\datasets\IEMOCAP...
1,G:\Arete\projects\SER_project\datasets\IEMOCAP...
2,G:\Arete\projects\SER_project\datasets\IEMOCAP...
3,G:\Arete\projects\SER_project\datasets\IEMOCAP...
4,G:\Arete\projects\SER_project\datasets\IEMOCAP...


In [9]:
df = pd.read_csv(df_path)
print(df.shape)
df.head()

(10039, 4)


,main_emotion,start_time,end_time,titre
0,neu,6.2901,8.2357,Ses01F_impro01_F000
1,neu,10.0100,11.3925,Ses01F_impro01_F001
2,neu,14.8872,18.0175,Ses01F_impro01_F002
3,neu,19.2900,20.7875,Ses01F_impro01_F003
4,neu,21.3257,24.7400,Ses01F_impro01_F004


In [10]:
Iemocap_df['main_emotion'] = df['main_emotion'][:151]
Iemocap_df

,name,main_emotion
0,G:\Arete\projects\SER_project\datasets\IEMOCAP...,neu
1,G:\Arete\projects\SER_project\datasets\IEMOCAP...,neu
2,G:\Arete\projects\SER_project\datasets\IEMOCAP...,neu
3,G:\Arete\projects\SER_project\datasets\IEMOCAP...,neu
4,G:\Arete\projects\SER_project\datasets\IEMOCAP...,neu
...,...,...
146,G:\Arete\projects\SER_project\datasets\IEMOCAP...,fru
147,G:\Arete\projects\SER_project\datasets\IEMOCAP...,fru
148,G:\Arete\projects\SER_project\datasets\IEMOCAP...,ang
149,G:\Arete\projects\SER_project\datasets\IEMOCAP...,ang


In [14]:
dic = {'neu':'n','fru':'f', 'ang':'a', 'sad':'s', 'hap':'h', 'exc':'e', 'sur':'z', 'fea':'r', 'oth':'o','dis':'d'}
df['main_emotion'] = df['main_emotion'].map(dic)
df

,main_emotion,start_time,end_time,titre
0,n,6.2901,8.2357,Ses01F_impro01_F000
1,n,10.0100,11.3925,Ses01F_impro01_F001
2,n,14.8872,18.0175,Ses01F_impro01_F002
3,n,19.2900,20.7875,Ses01F_impro01_F003
4,n,21.3257,24.7400,Ses01F_impro01_F004
...,...,...,...,...
10034,a,236.5700,244.8300,Ses05M_script03_2_M041
10035,a,244.8400,246.5800,Ses05M_script03_2_M042
10036,a,246.5900,248.8300,Ses05M_script03_2_M043
10037,a,255.8600,260.3300,Ses05M_script03_2_M044


In [15]:
df['main_emotion'].unique()

array(['n', 'f', 'a', 's', 'h', 'e', 'z', 'r', 'o', 'd'], dtype=object)

In [24]:
emo_every_ms = .5
batch_size = 1
record_length = 40.0 # sec
nmerged_vectors = 25 # 500 ms


In [34]:
df.loc[(df['start_time']>40.) & (df['end_time']<80.)]

,main_emotion,start_time,end_time,titre
7,f,46.58,52.1900,Ses01F_impro01_F007
8,f,56.16,58.8225,Ses01F_impro01_F008
9,f,61.87,65.9700,Ses01F_impro01_F009
10,f,66.42,69.3400,Ses01F_impro01_F010
22,f,41.23,46.9800,Ses01F_impro01_M006
...,...,...,...,...
10002,a,50.70,52.9500,Ses05M_script03_2_M009
10003,n,54.15,56.0400,Ses05M_script03_2_M010
10004,n,57.25,59.1300,Ses05M_script03_2_M011
10005,f,62.27,65.9300,Ses05M_script03_2_M012


In [75]:


def detect_labels(time,sub_df):
  # times between which to extract the wave from
  # duration = sub_df.iloc[-1,2]
  start = 0.0 # seconds
  end = time # seconds
  data_dic = {}
  idx=0

  wav_name = sub_df['titre'][0][:-5]
  source_path = 'G:\Arete\projects\SER_project\datasets\IEMOCAP\Iemocap_audio\iemocap_audio\long_wav\\'+wav_name+'.wav'
  
  with wave.open(source_path, "rb") as infile:
    # get file data
    nchannels = infile.getnchannels()
    sampwidth = infile.getsampwidth()
    framerate = infile.getframerate()
    nframes = infile.getnframes()
    duration = nframes/framerate

    while end < duration - time*.3:
      chunk_df = sub_df[(sub_df['start_time']>=start) & (sub_df['end_time']<=end)]
      data_dic[wav_name+'_'+str(idx)+'.wav'] = ''.join(chunk_df['main_emotion'])
      
      infile.setpos(int(start * framerate))
      # extract data
      data0 = infile.readframes(int((end - start) * framerate))

      # write the extracted data to a new file
      destination_path = 'G:\Arete\projects\SER_project\\finetune_s2t_model\iemocap_40s_waves\\'+wav_name+'_'+str(idx)+'.wav'
      with wave.open(destination_path, 'w') as outfile:
          outfile.setnchannels(nchannels)
          outfile.setsampwidth(sampwidth)
          outfile.setframerate(framerate)
          outfile.setnframes(int(len(data0) / sampwidth))
          outfile.writeframes(data0)

      start = end
      end += time
      idx+=1
  # print(len(emotion_list),duration)
  return data_dic


In [76]:
data_dict = {}
for i,path in enumerate(Iemocap_df['name'][:3]):
  record_name = path.split(sep='\\')[-1]
  sub_records = df[df['titre'].str.contains(record_name[:-4])]
  sub_records = sub_records.sort_values('start_time',ignore_index=True)
  # print(record_name,sub_records.iloc[:,:]) # record duration
  data_dict.update(detect_labels(record_length,sub_records))
  # print(sub_records.iloc[-1,2],len(Iemocap_df['main_emotion'][i]))

data = pd.DataFrame(list(data_dict.items()), columns=['name','emo_script'])
data

,name,emo_script
0,Ses01F_impro01_0.wav,nfnfnfnnfnff
1,Ses01F_impro01_1.wav,ffffffff
2,Ses01F_impro01_2.wav,faaffnaf
3,Ses01F_impro02_0.wav,sssssn
4,Ses01F_impro02_1.wav,sfssnss
5,Ses01F_impro02_2.wav,ssssasfs
6,Ses01F_impro02_3.wav,ssssns
7,Ses01F_impro02_4.wav,nssnnssf
8,Ses01F_impro03_0.wav,nhnhzheeehhehn
9,Ses01F_impro03_1.wav,hnhnheenhhhhehe


In [77]:
data.to_csv('G:\Arete\projects\SER_project\\finetune_s2t_model\\iemocap_40s_data.csv',index=False)

In [78]:
dataset = pd.read_csv('G:\Arete\projects\SER_project\\finetune_s2t_model\\iemocap_40s_data.csv')

In [9]:
source_path = 'G:\Arete\projects\SER_project\datasets\IEMOCAP\Iemocap_audio\iemocap_audio\IEMOCAP_wav\\Ses01F_impro01_F000.wav'

with wave.open(source_path, "rb") as infile:
# get file data
    nchannels = infile.getnchannels()
nchannels

1